In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("DN4_1_podatki.csv")
X = data.drop("Q", axis=1)
y = data[["Q"]]

In [2]:
data

,Q,Tw,Ta,theta,eta
0,1.711929,53.005339,10.788147,0.321921,0.349083
1,0.067135,20.761340,12.225502,2.705824,0.821069
2,0.034759,39.546739,35.892183,1.873776,0.756082
3,0.417783,33.972185,21.385801,2.089589,0.516134
4,1.667614,50.092173,15.254621,0.328350,0.083327
...,...,...,...,...,...
495,1.327393,54.081114,30.039247,1.153132,0.708757
496,0.053967,42.933431,39.799381,1.572222,0.022975
497,0.707523,47.145488,30.707064,2.093919,0.684035
498,0.062350,39.405105,32.647363,0.311168,0.053085


In [3]:
from pysr import PySRRegressor



In [4]:
model = PySRRegressor(
    niterations=100,  # < Increase me for better results
    #populations=30,
    #population_size=50,
    # ^ Slightly larger populations, for greater diversity.
    binary_operators=["+", "-", "*", "/"],
    unary_operators=[
        "cos",
        "sin",
    ],
    #timeout_in_seconds=60,
    #complexity_of_constants=0.2,
    #weight_randomize=0.1,
    #maxsize=30,
    maxdepth=9,
    procs=0,
    multithreading=False,
    random_state=0,
    deterministic=True
)

model.fit(X,y)
print(model.score(X,y))
model.sympy()

/home/user/programming/nsu/dn4/.venv/lib/python3.11/site-packages/pysr/sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/user/programming/nsu/dn4/.venv/lib/python3.11/site-packages/pysr/sr.py:1364: FutureWarning: Index.is_object is deprecated.Use pandas.api.types.is_object_dtype instead
  if X.columns.is_object() and X.columns.str.contains(" ").any():


0.9567841076185941

best 0.9874403551233546